In [ ]:
!nvidia-smi

In [ ]:
!pip install langchain_google_vertexai

In [ ]:
!pip install langchain_community

In [ ]:
!pip install langchain_experimental

In [ ]:
!pip install neo4j

In [ ]:
!pip install "unstructured[all-docs]" 

In [ ]:
!pip install pillow-heif

In [ ]:
!pip install unstructured_inference

In [ ]:
!pip install pytesseract
!pip install tesseract-ocr

In [ ]:
pip install tiktoken

In [21]:
!pip install json-repair

In [111]:
from pathlib import Path
from typing import List, Optional
from langchain_google_vertexai import (
    VertexAI,
    ChatVertexAI,
    VertexAIEmbeddings,
    VectorSearchVectorStore,
)
from google.cloud import aiplatform
from langchain_community.vectorstores import Neo4jVector
from langchain import LLMChain
from langchain_community.document_loaders import TextLoader
from langchain_community.graphs import Neo4jGraph
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_text_splitters import TokenTextSplitter
from langchain_experimental.text_splitter import SemanticChunker

from neo4j.exceptions import ClientError
import os
from vertexai.generative_models import GenerativeModel, Image
from vertexai.language_models import TextEmbeddingModel, TextEmbeddingInput
from langchain_google_vertexai import ChatVertexAI, HarmBlockThreshold, HarmCategory
from unstructured.partition.pdf import partition_pdf
from langchain_text_splitters import CharacterTextSplitter
from langchain_core.messages import AIMessage, HumanMessage
from langchain.prompts import PromptTemplate

from typing import List, Tuple
import base64
from langchain_google_vertexai import ChatVertexAI
from langchain_core.runnables import ConfigurableField, RunnableParallel, RunnableLambda
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_community.graphs.graph_document import GraphDocument

In [2]:
url = "bolt://10.113.8.4:8085"
username ="neo4j"
password = "password"
os.environ["NEO4J_URI"] = url
os.environ["NEO4J_USERNAME"] = username
os.environ["NEO4J_PASSWORD"] = password

In [3]:
PROJECT_ID = "edl-idaas-rnd-platform-d5ae"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
aiplatform.init(project=PROJECT_ID, location=LOCATION)

In [30]:
llm = ChatVertexAI(
    model="gemini-1.5-flash-001",
    temperature=0,
    max_tokens=None,
    max_retries=6,
    stop=None,
    # other params...
)
ollamallm = ChatOllama(base_url='http://10.113.8.4:8086',
    model="llama3:latest", temperature=0)

In [5]:
pdf_folder_path = "/home/sridhanya_ganapathi_team_neustar/Assignment/data/images"
pdf_file_name = "/home/sridhanya_ganapathi_team_neustar/Assignment/data/Health-Guard-Brochure-print.pdf"
Pdf_title="Health-Guard-Brochure-print.pdf"

In [6]:
image_pdf_folder_path = "/home/sridhanya_ganapathi_team_neustar/Assignment/data/images"
pdf_file_name = "/home/sridhanya_ganapathi_team_neustar/Assignment/data/Health-Guard-Brochure-print.pdf"

# Extract images, tables, and chunk text from a PDF file.
raw_pdf_elements = partition_pdf(
    filename=pdf_file_name,
    extract_images_in_pdf=True,
    infer_table_structure=True,
    chunking_strategy="by_title",
    max_characters=4000,
    new_after_n_chars=3800,
    combine_text_under_n_chars=2000,
    image_output_dir_path=pdf_folder_path,
    extract_image_block_types=["Image", "Table"],
    extract_image_block_output_dir=image_pdf_folder_path,
    extract_image_block_to_payload=False
)

# Categorize extracted elements from a PDF into tables and texts.
tables = []
texts = []
for element in raw_pdf_elements:
    if "unstructured.documents.elements.Table" in str(type(element)):
        tables.append(str(element))
    elif "unstructured.documents.elements.CompositeElement" in str(type(element)):
        texts.append(str(element))


Some weights of the model checkpoint at microsoft/table-transformer-structure-recognition were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
len(texts)

36

In [8]:
MODEL_NAME = "gemini-1.0-pro-vision"


# Generate summaries of text elements
def generate_text_summaries(
    texts: List[str], tables: List[str], summarize_texts: bool = False
) -> Tuple[List, List]:
    """
    Summarize text elements
    texts: List of str
    tables: List of str
    summarize_texts: Bool to summarize texts
    """

    # Prompt
    prompt_text = """You are an assistant tasked with summarizing tables and text for retrieval. \
    These summaries will be embedded and used to retrieve the raw text or table elements. \
    Give a ellaborate and concise summary of the table or text that is well optimized for retrieval. Table or text: {element} """
    prompt = PromptTemplate.from_template(prompt_text)
    empty_response = RunnableLambda(
        lambda x: AIMessage(content="Error processing document")
    )
    # Text summary chain
    model = VertexAI(
        temperature=0, model_name=MODEL_NAME, max_output_tokens=1024
    ).with_fallbacks([empty_response])
    summarize_chain = {"element": lambda x: x} | prompt | model | StrOutputParser()

    # Initialize empty summaries
    text_summaries = []
    table_summaries = []

    # Apply to text if texts are provided and summarization is requested
    if texts:
        if summarize_texts:
            text_summaries = summarize_chain.batch(texts, {"max_concurrency": 1})
        else:
            text_summaries = texts

    # Apply to tables if tables are provided
    if tables:
        table_summaries = summarize_chain.batch(tables, {"max_concurrency": 1})

    return text_summaries, table_summaries


# Get text, table summaries
text_summaries, table_summaries = generate_text_summaries(
    texts, tables, summarize_texts=True
)

In [9]:
def encode_image(image_path):
    """Getting the base64 string"""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")


def image_summarize(img_base64, prompt):
    """Make image summary"""
    model = ChatVertexAI(model_name="gemini-pro-vision", max_output_tokens=1024)

    msg = model(
        [
            HumanMessage(
                content=[
                    {"type": "text", "text": prompt},
                    {
                        "type": "image_url",
                        "image_url": {"url": f"data:image/png;base64,{img_base64}"},
                    },
                ]
            )
        ]
    )
    return msg.content


def generate_img_summaries(path):
    """
    Generate summaries and base64 encoded strings for images
    path: Path to list of .jpg files extracted by Unstructured
    """

    # Store base64 encoded images
    img_base64_list = []

    # Store image summaries
    image_summaries = []

    # Prompt
    prompt = """You are an assistant tasked with summarizing images for retrieval. \
    These summaries will be embedded and used to retrieve the raw image. \
    Give a concise summary of the image that is well optimized for retrieval.
    If it's a table, extract all elements of the table.
    If it's a graph, explain the findings in the graph.
    Do not include any numbers that are not mentioned in the image.
    """

    # Apply to images
    for img_file in sorted(os.listdir(path)):
        if img_file.endswith(".jpg"):
            img_path = os.path.join(path, img_file)
            base64_image = encode_image(img_path)
            img_base64_list.append(base64_image)
            image_summaries.append(image_summarize(base64_image, prompt))

    return img_base64_list, image_summaries


# Image summaries
img_base64_list, image_summaries = generate_img_summaries(image_pdf_folder_path)

/opt/conda/envs/llm/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


In [10]:
combined_all_texts = text_summaries + tables + image_summaries

In [11]:
combined_text = ".".join(combined_all_texts)

In [23]:

# Optional: Enforce a specific token size for texts
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000, chunk_overlap=0
)

In [24]:
texts_4k_token = text_splitter.split_text(combined_text)

Created a chunk of size 2395, which is longer than the specified 1000
Created a chunk of size 1125, which is longer than the specified 1000


In [25]:
len(texts_4k_token)

12

In [26]:
docs = text_splitter.create_documents(texts_4k_token)
len(docs)

12

In [65]:
llm_transformer = LLMGraphTransformer(llm=ollamallm, strict_mode=False)

In [72]:
graph_documents = []

In [73]:
ngraph = Neo4jGraph()

In [74]:
try:
    for i in range(0, len(docs)):
        graph = llm_transformer.process_response(docs[i])
        graph_documents.extend(graph)
except Exception as e:
    print(f"Error processing document: {e}")

In [75]:
# Ensure graph_documents contains the correct objects
processed_graph_documents = []
for document in graph_documents:
    if isinstance(document, tuple):
        node = document[1]  # Assuming the Node object is the second element in the tuple
        processed_graph_documents.append(node)
    else:
        processed_graph_documents.append(document)


In [87]:
graph_document = GraphDocument(nodes=processed_graph_documents[0], relationships=processed_graph_documents[1], source=processed_graph_documents[2])


In [88]:

# Add graph documents
ngraph.add_graph_documents(
    [graph_document],
    baseEntityLabel=True,
    include_source=False
)

In [89]:
# Embeddings & LLM models
# embeddings = OpenAIEmbeddings()
embedding_dimension = 768
# llm = ChatOpenAI(temperature=0)

# Loading Gemini Pro Vision Model
multimodal_model = GenerativeModel("gemini-1.0-pro-vision")
# Initializing embedding model
EMBEDDING_MODEL =VertexAIEmbeddings(model_name="textembedding-gecko@003", dimension=768) #TextEmbeddingModel.from_pretrained("textembedding-gecko@003")

# Loading Gemini Pro Model
llm = ChatVertexAI(
    model_name="gemini-1.0-pro-vision",
    safety_settings={
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE
    },
)


In [90]:

parent_splitter = TokenTextSplitter(chunk_size=1048, chunk_overlap=24)
child_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=24)
parent_documents = parent_splitter.split_documents(docs)

In [91]:
graph = Neo4jGraph()

In [93]:
for i, parent in enumerate(parent_documents):
    child_documents = child_splitter.split_documents([parent])
    print(len(parent.page_content))
    params = {
        "parent_text": parent.page_content,
        "parent_id": i,
        "parent_embedding": EMBEDDING_MODEL.embed_query(parent.page_content), #EMBEDDING_MODEL.get_embeddings([parent.page_content])[0].values,
        "children": [
            {
                "text": c.page_content,
                "id": f"{i}-{ic}",
                "embedding": EMBEDDING_MODEL.embed_query(c.page_content) #EMBEDDING_MODEL.get_embeddings([c.page_content])[0].values,
            }
            for ic, c in enumerate(child_documents)
        ],
    }
     # Ingest data
    graph.query(
        """
    MERGE (p:Parent {id: $parent_id})
    SET p.text = $parent_text
    WITH p
    CALL db.create.setVectorProperty(p, 'embedding', $parent_embedding)
    YIELD node
    WITH p 
    UNWIND $children AS child
    MERGE (c:Child {id: child.id})
    SET c.text = child.text
    MERGE (c)<-[:HAS_CHILD]-(p)
    WITH c, child
    CALL db.create.setVectorProperty(c, 'embedding', child.embedding)
    YIELD node
    RETURN count(*)
    """,
        params,
    )
    # Create vector index for child
    try:
        graph.query(
            "CALL db.index.vector.createNodeIndex('parent_document', "
            "'Child', 'embedding', 768, 'cosine')",
            {},
        )
    except ClientError:  # already exists
        pass
    # Create vector index for parents
    try:
        graph.query(
            "CALL db.index.vector.createNodeIndex('typical_rag', "
            "'Parent', 'embedding', 768, 'cosine')",
            {},
        )
    except ClientError:  # already exists
        pass
# CALL db.index.vector.createNodeIndex('parent_document', 'Child', 'embedding', 768, 'cosine')

3660
3816
4625
4067
4012
4297
375
3301
4059
1437
2135
4179
502
3451
2054


In [96]:
# Ingest frequently asked questions  questions


class Questions(BaseModel):
    """Generating hypothetical questions about text."""

    questions: List[str] = Field(
        ...,
        description=(
            "Generated hypothetical questions based on  the information from the text"
        ),
    )


In [97]:

questions_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "human",
            (
                "Use the given format to generate frequently asked questions from the "
                "following input: {input}"
            ),
        ),
    ]
)
question_chain = questions_prompt | llm.with_structured_output(Questions)


In [98]:
question_chain = questions_prompt | llm.with_structured_output(Questions)

In [100]:
for i, parent in enumerate(parent_documents):
    resp = question_chain.invoke([parent.page_content])
    if (resp != None):
        questions = resp.questions
        params = {
            "parent_id": i,
            "questions": [
                {"text": q, "id": f"{i}-{iq}", "embedding": EMBEDDING_MODEL.embed_query(q)} #EMBEDDING_MODEL.get_embeddings([q])[0].values}
                for iq, q in enumerate(questions)
                if q
            ],
        }
        graph.query(
            """
        MERGE (p:Parent {id: $parent_id})
        WITH p
        UNWIND $questions AS question
        CREATE (q:Question {id: question.id})
        SET q.text = question.text
        MERGE (q)<-[:HAS_QUESTION]-(p)
        WITH q, question
        CALL db.create.setVectorProperty(q, 'embedding', question.embedding)
        YIELD node
        RETURN count(*)
        """,
            params,
        )
        # Create vector index #CALL db.index.vector.createNodeIndex
        try:
            graph.query(
                "CALL db.index.vector.createNodeIndex('hypothetical_questions', "
                "'Question', 'embedding', 768, 'cosine')",
                {},
            )
        except ClientError:  # already exists
            pass
#CALL db.index.vector.createNodeIndex('hypothetical_questions', 'Question', 'embedding', 768, 'cosine')

In [102]:


# Ingest summaries

summary_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "human",
            ("Generate a summary of the following input: {question}\n" "Summary:"),
        ),
    ]
)

summary_chain = summary_prompt | llm


In [103]:

for i, parent in enumerate(parent_documents):
    summary = summary_chain.invoke({"question": parent.page_content}).content
    params = {
        "parent_id": i,
        "summary": summary,
        "embedding": EMBEDDING_MODEL.embed_query(summary)# EMBEDDING_MODEL.get_embeddings([summary])[0].values,
    }
    graph.query(
        """
    MERGE (p:Parent {id: $parent_id})
    MERGE (p)-[:HAS_SUMMARY]->(s:Summary)
    SET s.text = $summary
    WITH s
    CALL db.create.setVectorProperty(s, 'embedding', $embedding)
    YIELD node
    RETURN count(*)
    """,
        params,
    )
    # Create vector index
    #CALL db.index.vector.createNodeIndex('typical_rag', 'Parent', 'embedding', 768, 'cosine')
    try:
        graph.query(
            "CALL db.index.vector.createNodeIndex('summary', "
            "'Summary', 'embedding', 768, 'cosine')",
            {},
        )
    except ClientError:  # already exists
        pass
# CALL db.index.vector.createNodeIndex('summary', 'Summary', 'embedding', 768, 'cosine')

In [144]:
intent_prompt_template = '''You are a helpful assistant and an expert in natural language processing and specialize in open set discovery intent classification. This task involves assigning textual utterances to specific intents,
                some of which are pre-defined (known) and others are not (unknown). \nAs an expert in open set discovery intent classification,
                your task is to provide a list of  pairs of utterances and intents that are the most difficult to classify. These pairs should consist of one utterance and its corresponding intent. Make sure that the final list contains high variety of intents. The hard samples you propose need to be extracted from the original list.
                given the following text: {Text}
                To select the most difficult pairs, consider the following criteria:
                1. Ambiguity: Choose utterances that can be interpreted in multiple ways, making it challenging to determine the correct intent.
                2. Contextual complexity: Select pairs where the intent is highly dependent on the context provided in the utterance or surrounding dialogue.
                3. Lack of explicit keywords: Include pairs with utterances that do not contain obvious keywords or explicit indicators of the intent.
                4. Similarities among intents: Include pairs where intents have overlapping or similar meanings, making it challenging to differentiate between them.

                Once you have selected the difficult pairs, and have acquired sufficient context about the problem, provide a detailed prompt for an AI language model to solve the task of open set intent discovery, maximizing the model's performance in the task. Provide effective guidelines about how to solve the task in the prompt. 

                EXAMPLES:
                
                Text : please book an appointment with a Dermatologist 
                Intent: book/create/appointment
                Text :  how much total coverage do I have left 
                Intent: coverage/calculate/pendingamount
        
                Else 
                Fallback Intent: query 
               '''

In [145]:
from langchain_google_vertexai import ChatVertexAI

cllm = ChatVertexAI(
    model="gemini-1.0-pro-vision",
    temperature=0,
    max_tokens=None,
    max_retries=6,
    stop=None,
    # other params...
)

In [146]:
input_text="please book an appointment with a Doctor"

In [147]:
intent_prompt = ChatPromptTemplate.from_template(intent_prompt_template)

In [148]:
chain = intent_prompt | ollamallm
chain.invoke(
    {
        "Text": input_text
    }
)

AIMessage(content='After analyzing the provided text, I\'ve extracted the most challenging pairs of utterances and intents that meet the criteria for ambiguity, contextual complexity, lack of explicit keywords, and similarities among intents. Here are some examples:\n\n1. Text: "I need to reschedule my appointment."\nIntent: book/create/appointment (or possibly query)\n\nThis pair is challenging because it can be interpreted as either a request to reschedule an existing appointment or a query about the possibility of rescheduling.\n\n2. Text: "What\'s the best way to get rid of acne?"\nIntent: query (or possibly treatment/consultation)\n\nThis pair is difficult because the utterance could be asking for general advice on how to treat acne, rather than seeking a specific consultation with a doctor.\n\n3. Text: "I\'m experiencing some weird symptoms."\nIntent: report/symptom (or possibly query)\n\nThis pair is challenging because the utterance could be reporting unusual symptoms or simply